## Import libraries

In [28]:
import pandas as pd
from openai import OpenAI
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players, teams
import plotly.express as px
pd.options.display.max_columns = None
client = OpenAI(api_key='YOUR_OPENAI_API_KEY')

## Get static data about players and teams (ID's, full name, city, state, etc.)

In [4]:
players_static = players.get_players()

In [11]:
pd.DataFrame(players_static).to_csv('players_static.csv', index=False)

In [7]:
teams_static = teams.get_teams()

In [12]:
pd.DataFrame(teams_static).to_csv('teams_static.csv', index=False)

In [39]:
players_static = pd.read_csv('players_static.csv')
teams_static = pd.read_csv('teams_static.csv')

In [41]:
players_static.sample(10)

,id,full_name,first_name,last_name,is_active
3393,77825,George Pearcy,George,Pearcy,False
1849,1538,Cedric Henderson,Cedric,Henderson,False
4077,22,Rik Smits,Rik,Smits,False
2829,77517,Paul McCracken,Paul,McCracken,False
3022,600006,Earl Monroe,Earl,Monroe,False
4324,78340,Mel Thurston,Mel,Thurston,False
3460,2565,Zoran Planinic,Zoran,Planinic,False
1782,203914,Gary Harris,Gary,Harris,True
42,203128,Furkan Aldemir,Furkan,Aldemir,False
1094,76577,Terry Dischinger,Terry,Dischinger,False


In [42]:
teams_static.sample(10)

,id,full_name,abbreviation,nickname,city,state,year_founded
13,1610612750,Minnesota Timberwolves,MIN,Timberwolves,Minnesota,Minnesota,1989
15,1610612752,New York Knicks,NYK,Knicks,New York,New York,1946
27,1610612764,Washington Wizards,WAS,Wizards,Washington,District of Columbia,1961
21,1610612758,Sacramento Kings,SAC,Kings,Sacramento,California,1948
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970
19,1610612756,Phoenix Suns,PHX,Suns,Phoenix,Arizona,1968
14,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946


In [43]:
players_static[players_static['full_name'].isin(['LeBron James', 'Michael Jordan'])]

,id,full_name,first_name,last_name,is_active
2111,2544,LeBron James,LeBron,James,True
2291,893,Michael Jordan,Michael,Jordan,False


## Get player career stats using the player's ID

In [24]:
lebron_james = playercareerstats.PlayerCareerStats(player_id=2544)
michael_jordan = playercareerstats.PlayerCareerStats(player_id=893)

In [33]:
lebron_james.get_data_frames()[0].head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120.0,622,1492,0.417,63,217,0.290,347,460,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388.0,795,1684,0.472,108,308,0.351,477,636,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361.0,875,1823,0.480,127,379,0.335,601,814,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190.0,772,1621,0.476,99,310,0.319,489,701,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027.0,794,1642,0.484,113,359,0.315,549,771,0.712,133,459,592,539,138,81,255,165,2250


In [34]:
michael_jordan.get_data_frames()[0].head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,893,1984-85,00,1610612741,CHI,22.0,82,82,3144.0,837,1625,0.515,9,52,0.173,630,746,0.845,167,367,534,481,196,69,291,285,2313
1,893,1985-86,00,1610612741,CHI,23.0,18,7,451.0,150,328,0.457,3,18,0.167,105,125,0.840,23,41,64,53,37,21,45,46,408
2,893,1986-87,00,1610612741,CHI,24.0,82,82,3281.0,1098,2279,0.482,12,66,0.182,833,972,0.857,166,264,430,377,236,125,272,237,3041
3,893,1987-88,00,1610612741,CHI,25.0,82,82,3311.0,1069,1998,0.535,7,53,0.132,723,860,0.841,139,310,449,485,259,131,252,270,2868
4,893,1988-89,00,1610612741,CHI,26.0,81,81,3255.0,966,1795,0.538,27,98,0.276,674,793,0.850,149,503,652,650,234,65,290,247,2633


## Create a prompt template and dynamically insert each player's data

In [70]:
for player_id in players_static['id'].sample(3):
    player_stats_df = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
    player_stats_df = pd.merge(
        player_stats_df,
        players_static,
        left_on='PLAYER_ID',
        right_on='id',
        how='left')
    player_stats_df = pd.merge(
        player_stats_df,
        teams_static,
        left_on='TEAM_ID',
        right_on='id',
        how='left')
    display(player_stats_df)

    # player_id = int(player_id.replace('player_', '').replace('.csv', ''))
    # t0 = time.time()
    try:
        df = player_stats_df[player_stats_df['PLAYER_ID'].eq(player_id)]
        if (df['FGM'].sum() > 0) and  (df['FGA'].sum() > 0):
            goals_percent = df['FGM'].sum() / df['FGA'].sum()
        else:
            goals_percent = 'unknown'
        if (df['FTM'].sum() > 0) and (df['FTA'].sum() > 0):
            freethrows_percent = df['FTM'].sum() / df['FTA'].sum() > 0
        else:
            freethrows_percent = 'unknown'
        d = dict(
            name = df['full_name_x'].iloc[0],
            teams = df['full_name_y'].drop_duplicates().dropna().tolist(),
            cities = df['city'].drop_duplicates().dropna().tolist(),
            states = df['state'].drop_duplicates().dropna().tolist(),
            active = df['is_active'].iloc[-1],
            start_season = df['SEASON_ID'].iloc[0],
            end_season = df['SEASON_ID'].iloc[-1],
            start_age = df['PLAYER_AGE'].iloc[0],
            end_age = df['PLAYER_AGE'].iloc[-1],
            games_played = df['GP'].sum(),
            minutes_played = df['MIN'].sum(),
            goals_attempted = df['FGA'].sum(),
            goals_made = df['FGM'].sum(),
            goals_percent = goals_percent,
            freethrows_attempted = df['FTA'].sum(),
            freethrows_made = df['FTM'].sum(),
            freethrows_percent = freethrows_percent,
            rebounds = df['REB'].sum(),
            rebounds_def = df['DREB'].sum(),
            rebounds_off = df['OREB'].sum(),
            assists = df['AST'].sum(),
            steals = df['STL'].sum(),
            blocks = df['BLK'].sum(),
            points = df['PTS'].sum(),
    )
        prompt = [
            {"role": "system",
             "content": """
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player."""},
             {"role": "user",
              "content": f"""
Please write and article for the player using the following details:
{d}"""}]
        print()
        for k, v in prompt[0].items():
            print(k, v)
        print()
        for k, v in d.items():
            print(k, v)
    except Exception as e:
        print(str(e))
        continue
        # completion = client.chat.completions.create(
        #   model=model,
        #   messages=[
        #     {"role": "system",
        #      "content": """You are a smart, detail-oriencted, keen NBA Basketball player analyst. \
        #      Please write an introductory text for a profile page of this Basketball player.
        #      Length: 500 - 800 words.
        #      please stick to the stats provided.
        #      Tone: should be interesting factual intriguing and inviting the user to dive \
        #      into the charts on the page to better get to know the player.
        #      """},
        #     {"role": "user",
        #      "content": f"""Please write and article for the player using the following details:
        #      {d}
        #      """}
        #   ]
        # )
    #     responses.append([player_id, completion])

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,id_x,full_name_x,first_name,last_name,is_active,id_y,full_name_y,abbreviation,nickname,city,state,year_founded
0,956,1996-97,00,1610612754,IND,21.0,72,21,1048.0,131,336,0.390,1,1,1.000,107,168,0.637,96,198,294,43,19,73,84,153,370,956,Erick Dampier,Erick,Dampier,False,1610612754,Indiana Pacers,IND,Pacers,Indiana,Indiana,1976
1,956,1997-98,00,1610612744,GSW,22.0,82,82,2656.0,352,791,0.445,0,2,0.000,267,399,0.669,272,443,715,94,39,139,175,281,971,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
2,956,1998-99,00,1610612744,GSW,23.0,50,50,1414.0,161,414,0.389,0,0,0.000,120,204,0.588,164,218,382,54,26,58,92,165,442,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
3,956,1999-00,00,1610612744,GSW,24.0,21,12,495.0,70,173,0.405,0,0,0.000,27,51,0.529,48,86,134,19,8,15,29,75,167,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
4,956,2000-01,00,1610612744,GSW,25.0,43,26,1042.0,126,314,0.401,0,2,0.000,67,126,0.532,97,153,250,59,17,58,82,123,319,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
5,956,2001-02,00,1610612744,GSW,26.0,73,46,1737.0,209,480,0.435,0,0,0.000,136,211,0.645,167,220,387,87,17,167,156,235,554,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
6,956,2002-03,00,1610612744,GSW,27.0,82,82,1977.0,259,522,0.496,0,2,0.000,155,222,0.698,248,295,543,58,27,154,112,242,673,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
7,956,2003-04,00,1610612744,GSW,28.0,74,74,2406.0,348,650,0.535,0,2,0.000,217,332,0.654,344,543,887,60,33,137,131,227,913,956,Erick Dampier,Erick,Dampier,False,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,956,2004-05,00,1610612742,DAL,29.0,59,56,1609.0,202,367,0.550,0,3,0.000,138,228,0.605,183,318,501,51,15,80,102,205,542,956,Erick Dampier,Erick,Dampier,False,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
9,956,2005-06,00,1610612742,DAL,30.0,82,36,1934.0,171,347,0.493,0,0,0.000,127,215,0.591,273,367,640,51,27,106,115,263,469,956,Erick Dampier,Erick,Dampier,False,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980



role system
content 
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player.

name Erick Dampier
teams ['Indiana Pacers', 'Golden State Warriors', 'Dallas Mavericks', 'Miami Heat', 'Atlanta Hawks']
cities ['Indiana', 'Golden State', 'Dallas', 'Miami', 'Atlanta']
states ['Indiana', 'California', 'Texas', 'Florida', 'Georgia']
active False
start_season 1996-97
end_season 2011-12
start_age 21.0
end_age 36.0
games_played 987
minutes_played 24002.0
goals_attempted 5595
goals_made 2784
goals_percent 0.4975871313672922
freethrows_attempted 2779
freethrows_made 1739
freethrows_percent True
rebounds 7005
rebounds_def 4282
rebounds_off 2723
assists 817
steals 330
blocks 1398
points 7309


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,id_x,full_name_x,first_name,last_name,is_active,id_y,full_name_y,abbreviation,nickname,city,state,year_founded
0,1619,1997-98,00,1610612743,DEN,23.0,28,0,225.0,22,65,0.338,5,14,0.357,19,25,0.76,3,16,19,28,7,1,15,22,68,1619,Kiwane Garris,Kiwane,Garris,False,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
1,1619,1999-00,00,1610612753,ORL,25.0,3,0,23.0,2,10,0.200,0,1,0.000,0,0,0.00,0,1,1,2,0,0,1,0,4,1619,Kiwane Garris,Kiwane,Garris,False,1610612753,Orlando Magic,ORL,Magic,Orlando,Florida,1989



role system
content 
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player.

name Kiwane Garris
teams ['Denver Nuggets', 'Orlando Magic']
cities ['Denver', 'Orlando']
states ['Colorado', 'Florida']
active False
start_season 1997-98
end_season 1999-00
start_age 23.0
end_age 25.0
games_played 31
minutes_played 248.0
goals_attempted 75
goals_made 24
goals_percent 0.32
freethrows_attempted 25
freethrows_made 19
freethrows_percent True
rebounds 20
rebounds_def 17
rebounds_off 3
assists 30
steals 7
blocks 1
points 72


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,id_x,full_name_x,first_name,last_name,is_active,id_y,full_name_y,abbreviation,nickname,city,state,year_founded
0,1629058,2018-19,00,1610612751,BKN,20.0,9,0,39.0,9,22,0.409,1,10,0.100,0,2,0.00,1,4,5,2,2,0,4,1,19,1629058,Dzanan Musa,Dzanan,Musa,False,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
1,1629058,2019-20,00,1610612751,BKN,21.0,40,0,487.0,64,172,0.372,19,78,0.244,45,60,0.75,21,65,86,43,15,1,36,30,192,1629058,Dzanan Musa,Dzanan,Musa,False,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976



role system
content 
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player.

name Dzanan Musa
teams ['Brooklyn Nets']
cities ['Brooklyn']
states ['New York']
active False
start_season 2018-19
end_season 2019-20
start_age 20.0
end_age 21.0
games_played 49
minutes_played 526.0
goals_attempted 194
goals_made 73
goals_percent 0.37628865979381443
freethrows_attempted 62
freethrows_made 45
freethrows_percent True
rebounds 91
rebounds_def 69
rebounds_off 22
assists 45
steals 17
blocks 1
points 211
